In [ ]:
def get_outlier(df,k=10,quantile=0.9,fact_quantile=5,fact_std=5,percent_error=0.05,zoom_in=True):
  outlier_tot=pd.DataFrame()
  df_temp=df.copy()
  for c in df_temp.columns:
    print("___________________________________________________________________")
    temp_col=pd.Series(df_temp[c])
    temp_col.dropna(inplace=True)
    if len(temp_col)==0:
      #no values for this parameter
      print("No values for "+c+'\n')
      continue
    if temp_col.value_counts().values[0]/len(temp_col)>0.5:
      #if more than half of the values are identical 
      temp_col=temp_col[temp_col!=temp_col.value_counts().index[0]]
      print("Constant values deleted for "+c)
    idx_outliers=[]#list of the position of outliers in the DataFrame (not the date index)
    values=neighbor_distances(temp_col,k)#compute neighbor distances
    sigma=np.std(values)
    m=np.mean(values)
    q=np.quantile(values,quantile)
    for i in range(10,len(values)-10):
      #we don't consider first and last data because their distance values are biased
      if abs(values[i])>m+fact_std*sigma:
        if abs(values[i])>fact_quantile*q:
          if is_surrounded(temp_col,i)==False:#there is not enough neighbors both superior and inferior
            idx_outliers.append(i)
    if len(idx_outliers)==0:
      print('No outliers for', c,'\n')
    elif len(idx_outliers)>percent_error*len(df):
      #if we find too much outliers, there is a problem
      print("Cant find outliers for "+c+" please check it manually\n")
    else:
      outliers=temp_col.iloc[idx_outliers]#get the outliers from their position in the DataFrame
      print(len(outliers)," outliers for "+c)
      print(outliers,'\n')
      outlier_tot=pd.concat([outlier_tot,outliers],axis=1)
      plt.figure(figsize=(20,10))
      plt.plot(df[(df[c].isin(outlier_tot[c])==False)][c],color='#009FE3',zorder=1)#plot data without outliers
      plt.scatter(outlier_tot.index,outlier_tot[c],color='#ED008B',zorder=2)#plot scatter of outliers
      plt.title('Outlier for '+c,size=20)
      plt.show()
      if zomm_in=True:
        for idx,j in enumerate(idx_outliers):
          min=j-100
          max=j+100
          if min<0:
            min=0
          if max>len(temp_col):
            max=len(temp_col)
          plt.figure(figsize=(20,10))
          l=np.append(np.arange(min,j),np.arange(j+1,max))
          plt.plot(temp_col.iloc[l],color='#009FE3',zorder=1)
          plt.scatter(outliers.index[idx],outliers.iloc[idx],color='#ED008B',zorder=2)
          plt.show()
  return outlier_tot



#Fonctions annexes

def is_surrounded(l,i):
  #return True if there is enough point both superior and inferior to l[i] in his neighborhood
  n=int(0.01*len(l))
  s=n/5
  sup=0
  inf=0
  for j in range(1,n):
    if i+j<len(l) and i+j>=0:
      if l[i+j]>l[i]:
        sup+=1
      elif l[i+j]<l[i]:
        inf+=1
      if l[i-j]>l[i]:
        sup+=1
      elif l[i-j]<l[i]:
        inf+=1
  if sup>s and inf>s:
    return True
  return False

def neighbor_distances(l,k=10):
  neihbors_dist=np.zeros(len(l))
  for i in range(0,len(l)):
    dist_g=[]
    dist_d=[]
    for j in range(1,k+1):
      if i-j>0:
        d_gauche=abs(l[i]-l[i-j])
        dist_g.append(d_gauche)
      if i+k<len(l):
        d_droite=abs(l[i]-l[i+j])
        dist_d.append(d_droite)
    if dist_g==[]:
      d=np.mean(dist_d)
    elif dist_d==[]:
      d=np.mean(dist_g)
    else:
      d=min(np.mean(dist_g),np.mean(dist_d))
    neihbors_dist[i]=d
  return neihbors_dist